In [10]:
import matplotlib.pyplot as plt
from time import sleep
import torch.nn.functional as F
from fastai.vision import *
from fastai import *
from torch import optim
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

In [11]:
class Dataset():
    def __init__(self, x, y): self.x,self.y = x,y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i],self.y[i]

    
def get_dataloaders(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))

def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

In [19]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s): return (x-m)/s

In [20]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,64
c = y_train.max().item()+1
x_train.shape

torch.Size([50000, 784])

In [21]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c
        
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [22]:
data = DataBunch(*get_dataloaders(train_ds, valid_ds, bs), c)

In [23]:
#export
def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c))
    return model, optim.SGD(model.parameters(), lr=lr)

In [24]:
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data

In [25]:
loss_func = F.cross_entropy

In [29]:
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model=model
        self.opt=opt
        self.loss_func=loss_func
        self.data=data
        
    def fit(self, epochs):
        for epoch in range(epochs):
            self.model.train()
            for xb,yb in learn.data.train_dl:
                loss = self.loss_func(learn.model(xb), yb)
                loss.backward()
                self.opt.step()
                self.opt.zero_grad()

            self.model.eval()
            with torch.no_grad():
                tot_loss,tot_acc = 0.,0.
                for xb,yb in self.data.valid_dl:
                    pred = self.model(xb)
                    tot_loss += self.loss_func(pred, yb)
                    tot_acc  += accuracy (pred,yb)
            nv = len(self.data.valid_dl)
            print(epoch, tot_loss/nv, tot_acc/nv)
        return tot_loss/nv, tot_acc/nv

In [30]:
learn = Learner(*get_model(data), loss_func, data)

In [31]:
loss,acc = learn.fit(1)

0 tensor(0.1647) tensor(0.9509)


## Callback

Next, we will do two things:

1. We refactor the fit function, with the inner loop contents factored out:

```python
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()

def fit():
    for epoch in range(epochs):
        for b in train_dl: 
            one_batch(*b)
```

2. Add callbacks so we can remove complexity from loop, and make it flexible.

Then, we have refactored *Learner* class

In [32]:
class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model=model
        self.opt=opt
        self.loss_func=loss_func
        self.data=data

    def one_batch(self, xb, yb, cbh):
        self.xb, self.yb = xb, yb
        if cbh.begin_batch(xb,yb): return
        self.pred = self.model(xb)
        if cbh.begin_loss(): return
        self.loss = self.loss_func(self.model(xb), yb)
        if cbh.begin_backward(loss) or not self.in_train: return
        loss.backward()
        if cbh.after_backward(): return
        self.opt.step()
        if cbh.after_step(): return
        self.opt.zero_grad()

    def all_batches(self, dl, cbh):
        self.iters = len(dl)
        for xb, yb in dl:
            if self.stop: break
            self.one_batch(xb, yb, cbh)
            cbh.after_batch(): 
        self.stop = False

    def fit(self, epochs, cbh):
        self.epochs = epochs
        
        try:
            if cbhs.begin_fit(): return
            for epoch in range(epochs):
                self.epoch = epoch
                if not cbh.begin_epoch(epoch): 
                    self.all_batches(self.data.train_dl, cbh)

                with torch.no_grad(): 
                    if not cbh.begin_validate():
                        self.all_batches(self.data.valid_dl, cbh)
                if cbh.after_epoch(): 
                    break
        finally:
            cbh.after_fit()
            

We have a simple **Callback** class

In [36]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []

    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
        for cb in self.cbs: res = res and cb.begin_fit(learn)
        return res

    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
        for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
        for cb in self.cbs: res = res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res = res and cb.after_epoch()
        return res
    
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = self.in_train
        for cb in self.cbs: res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res = res and cb.after_step()
        return res
    
    def do_stop(self):
        try:     return self.learn.stop
        finally: self.learn.stop = False

In [ ]:
class CallbackHandler():
    def __init__(self,cbs=None):
        self.cbs = cbs if cbs else []

    def begin_fit(self, learn):
        self.learn,self.in_train = learn,True
        learn.stop = False
        res = True
#         for cb in self.cbs: res = res and cb.begin_fit(learn)
        self('begin_fit')
        return res

    def after_fit(self):
        res = not self.in_train
#         for cb in self.cbs: res = res and cb.after_fit()
        self('after_fit')
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train=True
        res = True
#         for cb in self.cbs: res = res and cb.begin_epoch(epoch)
        self('begin_epoch')
        return res

    def begin_validate(self):
        self.learn.model.eval()
        self.in_train=False
        res = True
#         for cb in self.cbs: res = res and cb.begin_validate()
        self('begin_validate')
        return res

    def after_epoch(self):
        res = True
#         for cb in self.cbs: res = res and cb.after_epoch()
        self('after_epoch')
        return res
    
    def begin_batch(self, xb, yb):
        res = True
#         for cb in self.cbs: res = res and cb.begin_batch(xb, yb)
        self('begin_batch')
        return res

    def after_loss(self, loss):
        res = self.in_train
#         for cb in self.cbs: res = res and cb.after_loss(loss)
        self('after_loss')
        return res

    def after_backward(self):
        res = True
#         for cb in self.cbs: res = res and cb.after_backward()
        self('after_backward')
        return res

    def after_step(self):
        res = True
#         for cb in self.cbs: res = res and cb.after_step()
        self('after_step')
        return res
    
    def do_stop(self):
        try:     return self.learn.stop
        finally: self.learn.stop = False

    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None)
            if f and f(): return True
        return False

In [ ]:
class Callback():
    def begin_fit(self,):
        self.learn = learn
        return True
    def after_fit(self): return True
    def begin_epoch(self, epoch):
        self.epoch=epoch
        return True
    def begin_validate(self): return True
    def after_epoch(self): return True
    def begin_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self): return True
    def after_step(self): return True

In [102]:
class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs

    @property
    def opt(self):       return self.learn.opt
    @property
    def model(self):     return self.learn.model
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def data(self):      return self.learn.data

    def one_batch(self, xb, yb):
        self.xb,self.yb = xb,yb
        if self('begin_batch'): return
        self.pred = self.model(self.xb)
        if self('after_pred'): return
        self.loss = self.loss_func(self.pred, self.yb)
        kk = self('after_loss')
        print("kk", kk)
        if kk or not self.in_train: return
        self.loss.backward()
        if self('after_backward'): return
        self.opt.step()
        if self('after_step'): return
        self.opt.zero_grad()

    def all_batches(self, dl):
        self.iters = len(dl)
        for xb,yb in dl:
            if self.stop: break
            self.one_batch(xb, yb)
            self('after_batch')
        self.stop=False

    def fit(self, epochs, learn):
        self.epochs,self.learn = epochs,learn

        try:
            for cb in self.cbs: cb.set_runner(self)
            if self('begin_fit'): return
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.data.train_dl)

                with torch.no_grad(): 
                    if not self('begin_validate'): self.all_batches(self.data.valid_dl)
                if self('after_epoch'): break
            
        finally:
            self('after_fit')
            self.learn = None

    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None)
#             print(f, cb_name)
            is_true = f and f()
            print("is_true", is_true)
            if is_true: return True
        return False

In [103]:
#export
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False
        
class AvgStats():
    def __init__(self, metrics, in_train): self.metrics,self.in_train = listify(metrics),in_train
    
    def reset(self):
        self.tot_loss,self.count = 0.,0
        self.tot_mets = [0.] * len(self.metrics)
        
    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]
    
    def __repr__(self):
        if not self.count: return ""
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"

    def accumulate(self, run):
        bn = run.xb.shape[0]
        self.tot_loss += run.loss * bn
        self.count += bn
        for i,m in enumerate(self.metrics):
            self.tot_mets[i] += m(run.pred, run.yb) * bn

class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
        
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
        
    def after_loss(self):
        print("AvgStatsCallback: after_loss")
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
    
    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)

In [104]:
learn = Learner(*get_model(data), loss_func, data)

In [105]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [106]:
run.fit(1, learn)

is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_t

is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_t

is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_t

is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_t

is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback:

...

is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_t

is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_t

is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true 

is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_t

is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
is_true None
AvgStatsCallback: after_loss
is_true None
kk False
is_true None
is_true None
is_true None
is_true None
is_tru